In [42]:
import pickle
import logging 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)
from sklearn.inspection import permutation_importance
from sklearn.utils import resample

# Corrected logging setup
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [55]:
import pandas as pd

# Cargar el archivo CSV sin asignar nombres a las columnas
df_raw = pd.read_csv("/workspaces/4geek_class/data/raw/qsar_aquatic_toxicity.csv", sep=',', header=None)

# Mostrar las primeras filas del DataFrame
print(df_raw.head())

# Mostrar el número de columnas
print("Número de columnas:", len(df_raw.columns))


                                       0
0      0;0;0;2.419;1.225;0.667;0;0;3.740
1      0;0;0;2.638;1.401;0.632;0;0;4.330
2   9.23;11;0;5.799;2.93;0.486;0;0;7.019
3  9.23;11;0;5.453;2.887;0.495;0;0;6.723
4  9.23;11;0;4.068;2.758;0.695;0;0;5.979
Número de columnas: 1


In [56]:
# Leer las primeras filas del archivo CSV como texto
with open("/workspaces/4geek_class/data/raw/qsar_aquatic_toxicity.csv", 'r') as file:
    lines = file.readlines()

# Mostrar las primeras 5 filas del archivo
for line in lines[:5]:
    print(line)


0;0;0;2.419;1.225;0.667;0;0;3.740

0;0;0;2.638;1.401;0.632;0;0;4.330

9.23;11;0;5.799;2.93;0.486;0;0;7.019

9.23;11;0;5.453;2.887;0.495;0;0;6.723

9.23;11;0;4.068;2.758;0.695;0;0;5.979



In [57]:
# Cargar el archivo CSV con un delimitador diferente
df_raw = pd.read_csv("/workspaces/4geek_class/data/raw/qsar_aquatic_toxicity.csv", sep=';', header=None)

# Asignar nombres a las columnas
df_raw.columns = ['TPSA', 'SAcc', 'H050', 'MLOGP', 'RDCH', 'GATS1p', 'nN', 'C040', 'LC50']

# Obtener información del DataFrame
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TPSA    546 non-null    float64
 1   SAcc    546 non-null    float64
 2   H050    546 non-null    int64  
 3   MLOGP   546 non-null    float64
 4   RDCH    546 non-null    float64
 5   GATS1p  546 non-null    float64
 6   nN      546 non-null    int64  
 7   C040    546 non-null    int64  
 8   LC50    546 non-null    float64
dtypes: float64(6), int64(3)
memory usage: 38.5 KB


In [58]:
df_raw = pd.read_csv("/workspaces/4geek_class/data/raw/qsar_aquatic_toxicity.csv", sep=',', header=None)
df_raw.columns = ['TPSA','SAcc','H050','MLOGP','RDCH','GATS1p','nN','C040','LC50']
df_raw.info()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 9 elements


### **3. Exploratory Data Analysis**
**Objective**: Summarize the main characteristics of the dataset using descriptive statistics and data visualization methods

In [60]:
df_train, df_test = train_test_split(
    df, test_size=0.15,
    random_state=2025
)
df_train, df_val = train_test_split(
    df_train, test_size=0.15,
    random_state=2025
)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_train.info()


NameError: name 'df' is not defined

In [45]:
df_train, df_test = train_test_split(
    df, test_size=0.15,
    random_state=2025
)
df_train, df_val = train_test_split(
    df_train, test_size=0.15
    random_state=2025
)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_train.info()


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1541689914.py, line 6)

In [49]:
df_train.describe().T

NameError: name 'df_train' is not defined

In [ ]:
df_train.hist()
plt.tight_layout()
plt.show

In [ ]:
sns.pairplot(df_train, diag_kind='kde', corner=True)
plt.show()

In [ ]:
sns.heatmap(df_train.corr(), vmin=-1, vmax=1, cmap='RdBu', annot=True)


### **4. Machine learning**
**Objective**: Create a model that learns from data to make predictions and generalize to unseen data, and thus perform tasks without explicit instructions

In [ ]:
x_train = df_train.drop(columns=['lc50'])
y_train = df_train['lc50']
x_val = df_val.drop(columns=['lc50'])
y_val = df_val['lc50']
x_test = df_test.drop(columns=['lc50'])
y_test = df_test['lc50']

In [ ]:
reg_knn_p = Pipeline([
    ('scaler', MinMaxScaler())
    ('knn', KNeighborsRegressor)
])
reg_knn_cv = GridSearchCV(
    reg_knn_p,
    param_grid={
        'knn__n_neighbors':np.arange(3, 31)
        'knn__weights':['uniform', 'distance']
        'knn__p':[1,2]
    },
    cv=5,
    scoring='r2',
    n_jobs=-1
)

In [ ]:
reg_knn_cv.fit(x_train, y_train)
reg_knn_cv.best_params_

In [ ]:
y_hat_val = reg_knn_cv.predict(x_val)
print(f'MSE:{mean_squared_error(y_val,y_hat_val):.2f}')
print(f'MAE:{mean_absolute_error(y_val,y_hat_val):.2f}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_val,y_hat_val)):.2f}')
print(f'R2: {r2_score(y_val,y_hat_val):.2f}')

In [ ]:
importance = permutation_importance(
    reg_knn_cv, X_train, y_train,
    n_repeats=25, random_state=2025
)
importance_df = pd.DataFrame(importance.importances.T, columns=X_train.columns)
ax = importance_df.plot.box(vert=False)
ax.axvline(x=0, linestyle'--', c='Tomato')
ax.grid(True)
plt.show()

In [ ]:
y_hat_test = reg_knn_cv.predict(x_test)
print(f'MSE:{mean_squared_error(y_val,y_hat_testl):.2f}')
print(f'MAE:{mean_absolute_error(y_val,y_hat_test):.2f}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_val,y_hat_test)):.2f}')
print(f'R2: {r2_score(y_val,y_hat_test):.2f}')

In [ ]:
def bootstrap_model(model, X_train, X_test, y_test, n_b=300):
    rmse_scores = []
    r2_scores = []
    result = {}
    for _ in range (n_b):
        idx = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_boot, y_boot = X_train.reindex(idx), y[idx]
        model.fit(X_boot, y_boot)
        y_temp = model.predict(x_test)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_temp)))
        r2_scores.append(r2_score(y_test, y_temp))
    result = {'RMSE':rmse_scores, 'R2':r2_scores}
    return result
result = bootstrap_model(reg_knn_cv, X_train, y_train, X_test, y_test, n_b=300)    

NameError: name 'reg_knn_cv' is not defined

In [ ]:
fig, ax = plt.Subplot(1, 2)
ax[0].boxplot(x=result['RMSE'], vert=False)
ax[0].grid(True)
ax[0].set_yticks([1], ['RMSE'])
ax[1].boxplot(x=result['R2'], vert=False)
ax[1].grid(True)
ax[1].set_yticks([1], ['R2'])
plt.suptitle('Model Intervals')
plt.tight_layout()
plt.show()

In [ ]:
ci_low_rmse, ci_high_rmse = np.percentile(result['RMSE'], [2.5, 97.5])
ci_low_r2, ci_high_r2 = np.percentile(result['R2'], [2.5, 97.5])
print(f"RMSE mean:{np.mean(result['RMSE'])}, CI:{ci_low_rmse}-{ci_high_rmse}")
print(f"R2 mean:{np.mean(result['R2'])}, CI:{ci_low_r2}-{ci_high_r2}")